In [1]:
import pandas as pd

In [2]:
prod_views_df = pd.read_csv('/home/ubuntu/recommendation_new/client', index_col=None)

In [3]:
prod_views_df.head(6)

,Unnamed: 0,cust_id,product_id,rating
0,0,3893506,com.gopaktor.subscription.v1.premium.1m,1
1,1,3793254,com.gopaktor.subscription.v1.premium.1m,1
2,2,6148794,com.gopaktor.subscription.v1.premium.1m,1
3,3,6149220,com.gopaktor.subscription.v4.premium.1m,1
4,4,6149432,com.gopaktor.subscription.v4.premium.1m,1
5,5,6149434,com.gopaktor.subscription.v4.premium.1m,1


In [4]:
prod_views_df.drop(prod_views_df.columns[[0]], axis=1, inplace = True)

In [5]:
prod_views_df.head()

,cust_id,product_id,rating
0,3893506,com.gopaktor.subscription.v1.premium.1m,1
1,3793254,com.gopaktor.subscription.v1.premium.1m,1
2,6148794,com.gopaktor.subscription.v1.premium.1m,1
3,6149220,com.gopaktor.subscription.v4.premium.1m,1
4,6149432,com.gopaktor.subscription.v4.premium.1m,1


In [7]:
prod_views_df.shape

(4428, 3)

In [8]:
prod_views_df_dup = prod_views_df.drop_duplicates()

In [9]:
prod_views_df_dup.head()

,cust_id,product_id,rating
0,3893506,com.gopaktor.subscription.v1.premium.1m,1
1,3793254,com.gopaktor.subscription.v1.premium.1m,1
2,6148794,com.gopaktor.subscription.v1.premium.1m,1
3,6149220,com.gopaktor.subscription.v4.premium.1m,1
4,6149432,com.gopaktor.subscription.v4.premium.1m,1


In [10]:
prod_views_df_dup.shape

(3232, 3)

In [11]:
from sklearn import preprocessing

In [12]:
user_id_le = preprocessing.LabelEncoder()
product_id_le = preprocessing.LabelEncoder()

In [13]:
user_id_le.fit(prod_views_df_dup.cust_id)

LabelEncoder()

In [14]:
product_id_le.fit(prod_views_df_dup.product_id)

LabelEncoder()

In [15]:
print('Number of unique users: ', str(len(user_id_le.classes_)))
print('Number of unique products: ', str(len(product_id_le.classes_)))

('Number of unique users: ', '3087')
('Number of unique products: ', '18')


In [16]:
n_prod_views_df = prod_views_df_dup

In [17]:
n_prod_views_df.cust_id = user_id_le.transform(prod_views_df_dup.cust_id)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [18]:
n_prod_views_df.product_id = product_id_le.transform(prod_views_df_dup.product_id)

In [19]:
n_prod_views_df.head()

,cust_id,product_id,rating
0,7,5,1
1,6,5,1
2,10,5,1
3,15,17,1
4,21,17,1


In [20]:
import findspark

In [21]:
findspark.init()

In [22]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [23]:
from pyspark.mllib.recommendation import ALS

In [24]:
sc = SparkContext(appName = "BuildProductRecommendations")

In [25]:
sqlContext = SQLContext(sc)

In [26]:
n_prod_views_rdd = sqlContext.createDataFrame(n_prod_views_df).rdd

In [27]:
n_prod_views_rdd.take(5)

[Row(cust_id=7, product_id=5, rating=1),
 Row(cust_id=6, product_id=5, rating=1),
 Row(cust_id=10, product_id=5, rating=1),
 Row(cust_id=15, product_id=17, rating=1),
 Row(cust_id=21, product_id=17, rating=1)]

In [28]:
training_rdd, validation_rdd, test_rdd = n_prod_views_rdd.randomSplit([6,2,2], 1345)

In [29]:
validation_for_predict_rdd = validation_rdd.map(lambda x : (x[0], x[1]))

In [30]:
test_for_predict_rdd = test_rdd.map(lambda x : (x[0], x[1]))

In [31]:
validation_for_predict_rdd.take(5)

[(7, 5), (21, 17), (30, 16), (74, 5), (113, 5)]

In [32]:
training_rdd.take(5)

[Row(cust_id=6, product_id=5, rating=1),
 Row(cust_id=10, product_id=5, rating=1),
 Row(cust_id=15, product_id=17, rating=1),
 Row(cust_id=22, product_id=17, rating=1),
 Row(cust_id=18, product_id=17, rating=1)]

In [33]:
print('Training RDD\n', training_rdd.take(5))
print('\nValidation for prediction RDD\n', validation_for_predict_rdd.take(5))
print('\nTest for Prediction RDD\n', test_for_predict_rdd.take(5))

('Training RDD\n', [Row(cust_id=6, product_id=5, rating=1), Row(cust_id=10, product_id=5, rating=1), Row(cust_id=15, product_id=17, rating=1), Row(cust_id=22, product_id=17, rating=1), Row(cust_id=18, product_id=17, rating=1)])
('\nValidation for prediction RDD\n', [(7, 5), (21, 17), (30, 16), (74, 5), (113, 5)])
('\nTest for Prediction RDD\n', [(24, 17), (28, 17), (33, 5), (48, 5), (16, 0)])


In [34]:
import math

In [35]:
seed = 49247
iterations = 10
lambdas = [0.01, 0.1]
ranks = [16]
alphas = [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 40.0, 80.0]
errors = [0 for x in range(len(alphas) * len(ranks) * len(lambdas))]
err_index = 0

In [36]:
for lambda_ in  lambdas:
    for rank in ranks:
        for alpha in alphas:
            model = ALS.trainImplicit(training_rdd, rank, seed=seed, iterations=iterations,
                                           lambda_ = lambda_, alpha = alpha)
            predictions = model.predictAll(validation_for_predict_rdd).map(lambda r : ((r[0], r[1]), r[2]))
            views_and_preds = validation_rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
            error = math.sqrt(views_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
            errors[err_index] = error
            err_index += 1
            print('For rank {0} at alpha: {1} and lambda: {2}, the RMSE is{3}'.format(rank, alpha, lambda_, error))

For rank 16 at alpha: 1.0 and lambda: 0.01, the RMSE is0.998049533979
For rank 16 at alpha: 2.0 and lambda: 0.01, the RMSE is0.998157365045
For rank 16 at alpha: 4.0 and lambda: 0.01, the RMSE is0.997613777809
For rank 16 at alpha: 8.0 and lambda: 0.01, the RMSE is0.997745917158
For rank 16 at alpha: 16.0 and lambda: 0.01, the RMSE is0.998684234656
For rank 16 at alpha: 32.0 and lambda: 0.01, the RMSE is0.999864149437
For rank 16 at alpha: 40.0 and lambda: 0.01, the RMSE is1.000183464
For rank 16 at alpha: 80.0 and lambda: 0.01, the RMSE is1.0015114913
For rank 16 at alpha: 1.0 and lambda: 0.1, the RMSE is0.997477095674
For rank 16 at alpha: 2.0 and lambda: 0.1, the RMSE is0.996503104175
For rank 16 at alpha: 4.0 and lambda: 0.1, the RMSE is0.994909823188
For rank 16 at alpha: 8.0 and lambda: 0.1, the RMSE is0.993604975053
For rank 16 at alpha: 16.0 and lambda: 0.1, the RMSE is0.993475141526
For rank 16 at alpha: 32.0 and lambda: 0.1, the RMSE is0.994710744387
For rank 16 at alpha: 40.

In [38]:
predictions = model.recommendProducts(6,10)

In [41]:
predictions

[Rating(user=6, product=5, rating=0.9945345734524925),
 Rating(user=6, product=0, rating=0.015250103498650303),
 Rating(user=6, product=9, rating=0.00250656088523743),
 Rating(user=6, product=12, rating=0.0010390971962856571),
 Rating(user=6, product=4, rating=0.0002958656597841225),
 Rating(user=6, product=2, rating=0.00012789750810215939),
 Rating(user=6, product=7, rating=2.8728915565323643e-05),
 Rating(user=6, product=15, rating=1.612854716900869e-05),
 Rating(user=6, product=11, rating=1.163886673749559e-05),
 Rating(user=6, product=13, rating=3.3029485211055754e-06)]

In [42]:
all_user = model.recommendProductsForUsers(15)

In [45]:
all_user.count()

1888